### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/snapshots/200K/movies.dat', 
                     delimiter='::', 
                     header=None, 
                     names=['movie_id', 'movie', 'genre'], 
                     dtype={'movie_id': object}, 
                     engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/snapshots/200K/ratings.dat', 
                      delimiter='::', 
                      header=None, 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      dtype={'user_id': object, 'movie_id': object, 'timestamp': object}, 
                      engine='python')

# https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat
# https://github.com/sidooms/MovieTweetings/blob/master/latest/movies.dat
# https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat
# https://github.com/sidooms/MovieTweetings/blob/master/latest/ratings.dat

# https://github.com/sidooms/MovieTweetings/tree/master/snapshots/200K

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
print(movies.shape)
print(movies.info())
print(movies.head())

(14732, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14732 entries, 0 to 14731
Data columns (total 3 columns):
movie_id    14732 non-null object
movie       14732 non-null object
genre       14621 non-null object
dtypes: object(3)
memory usage: 345.4+ KB
None
  movie_id                            movie                        genre
0  0000091       Le manoir du diable (1896)                 Short|Horror
1  0000628  The Adventures of Dollie (1908)                 Action|Short
2  0000833        The Country Doctor (1909)                  Short|Drama
3  0001223              Frankenstein (1910)          Short|Horror|Sci-Fi
4  0001740     The Lonedale Operator (1911)  Short|Drama|Romance|Western


In [3]:
print(movies.movie_id.nunique())
genre = movies.genre.str.split('|', expand = True).stack().unique()
print(genre)
print(len(genre))

14732
['Short' 'Horror' 'Action' 'Drama' 'Sci-Fi' 'Romance' 'Western' 'Crime'
 'Comedy' 'History' 'War' 'Biography' 'Sport' 'Adventure' 'Family'
 'Fantasy' 'Mystery' 'Thriller' 'Documentary' 'Animation' 'Music' 'Musical'
 'Film-Noir' 'Adult' 'News' 'Reality-TV']
26


In [4]:
# cell for work
print(reviews.shape)
print(reviews.info())
print(reviews.head())

(200000, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
user_id      200000 non-null object
movie_id     200000 non-null object
rating       200000 non-null int64
timestamp    200000 non-null object
dtypes: int64(1), object(3)
memory usage: 6.1+ MB
None
  user_id movie_id  rating   timestamp
0       1  0068646      10  1381620027
1       1  0113277      10  1379466669
2       2  0790636       7  1389963947
3       2  0816711       8  1379963769
4       2  1091191       7  1391173869


In [5]:
# more work
print(reviews.user_id.nunique())
print(reviews.rating.isnull().sum())
print('Average ratings: {}, minimum ratings: {}, and maximum: {}'.format(reviews.rating.mean(), 
             reviews.rating.min(), 
             reviews.rating.max()))

25011
0
Average ratings: 7.318705, minimum ratings: 0, and maximum: 10


In [6]:
# Use your findings to match each variable to the correct statement in the dictionary

# dict_sol1 = {
# 'The number of movies in the dataset': 
# 'The number of ratings in the dataset':
# 'The number of different genres':
# 'The number of unique users in the dataset':
# 'The number missing ratings in the reviews dataset':
# 'The average rating given across all ratings':
# 'The minimum rating given across all ratings':
# 'The maximum rating given across all ratings':
# }

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [7]:
# 1. Pull the date from the title and create new column
create_date = lambda x: x[-5:-1] if x[-1] == ')' else np.nan
movies['date'] = movies['movie'].apply(create_date)

# 2. Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)
    
# 3. Dummy column the genre with 1's and 0's
new_columns = {}
for value in genre:
    result = pd.Series(movies.genre.transform(lambda x: int(value.casefold() in str(x).casefold() )), 
                       index = movies.index)
    new_columns[value] = result

movies = pd.concat([movies, pd.DataFrame(new_columns)], axis = 1)


In [8]:
# and another
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Short,Horror,Action,...,Mystery,Thriller,Documentary,Animation,Music,Musical,Film-Noir,Adult,News,Reality-TV
0,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000628,The Adventures of Dollie (1908),Action|Short,1908,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0000833,The Country Doctor (1909),Short|Drama,1909,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0001223,Frankenstein (1910),Short|Horror|Sci-Fi,1910,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0001740,The Lonedale Operator (1911),Short|Drama|Romance|Western,1911,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# and a few more below, which you can use as necessary
# 4. Create a date out of time stamp
import datetime as dt

change_timestamp = lambda val: dt.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')
reviews['date'] = reviews['timestamp'].apply(change_timestamp)

reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0068646,10,1381620027,2013-10-12 23:20:27
1,1,0113277,10,1379466669,2013-09-18 01:11:09
2,2,0790636,7,1389963947,2014-01-17 13:05:47
3,2,0816711,8,1379963769,2013-09-23 19:16:09
4,2,1091191,7,1391173869,2014-01-31 13:11:09


In [10]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year